# Loan Data from Prosper Exploration
## by Jeff Mitchell

## Preliminary Wrangling

This dataset contains information on nearly 114,000 loans from Prosper. The dataset contains a lot of features and will require some wrangling before we can start analysing it.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

> Load in your dataset and describe its properties through the questions below.
Try and motivate your exploration goals through this section.

In [2]:
loans = pd.read_csv('prosperLoanData.csv')

In [3]:
# high-level overview of data shape and composition
print(loans.shape)
print(loans.dtypes)

(113937, 81)
ListingKey                              object
ListingNumber                            int64
ListingCreationDate                     object
CreditGrade                             object
Term                                     int64
LoanStatus                              object
ClosedDate                              object
BorrowerAPR                            float64
BorrowerRate                           float64
LenderYield                            float64
EstimatedEffectiveYield                float64
EstimatedLoss                          float64
EstimatedReturn                        float64
ProsperRating (numeric)                float64
ProsperRating (Alpha)                   object
ProsperScore                           float64
ListingCategory (numeric)                int64
BorrowerState                           object
Occupation                              object
EmploymentStatus                        object
EmploymentStatusDuration               float64


### What is the structure of your dataset?

There are 113,937 loans with 81 features for each loan. This is a lot of features and some are only relevant to certain periods of time (e.g. pre-2009, post-July 2009). These features include details such as Term, Loan status, Borrower rate, various scores and ratings, demographic features such as Occupation, Employment status, Home ownership, various credit scores and totals, income range and details specific to the loan.

### What is/are the main feature(s) of interest in your dataset?

I am most interested in finding out what features have the greatest impact on Loan Outcome Status, specifically on whether or not the loan is paid off.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

There are a number of features that I feel to be of interest in investigating Loan Outcome Status. These include the reason for the loan (ListingCategory), their employment status (EmploymentStatus), their home ownership status, their credit scores, number of delinquincies, their debt-to-income ratio (DebtToIncomeRatio), their monthly income, the ampunt of the loan (LoanOriginalAmount) and level of monthly payments (MonthlyLoanPayment). I expect that borrowers with higher debt-to-income ratios, lower monthly income and lower credit scores are less likely to complete their loan payments.

### Initial cleaning

A quick visual assessment looking at the data file shows that the data requires some initial cleaning before I can start analysing it. This will involve dropping unnecessary columns, cleaning column types and creating new columns to help with the analysis.

There are a lot of columns in the data which makes useful analysis difficult. I will start by reducing the dataframe to only the columns that are of interest to me for investigating the Loan Status Outcome. For example, LoanNumber can be used to identify individual loans and MemberKey can be used to identify individual users. This removes the need to hold ListingNumber, ListingKey and LoanKey which are all associated with a unique LoanNumber.

In [4]:
# Reduce the number of columns to just those that may be of interest
columns = ['Term', 'LoanStatus', 'BorrowerAPR', 'BorrowerRate', 'ListingCategory (numeric)', 'EmploymentStatus', 
          'EmploymentStatusDuration', 'IsBorrowerHomeowner', 'CreditScoreRangeLower', 'CreditScoreRangeUpper',
           'CurrentCreditLines', 'TotalCreditLinespast7years', 'OpenRevolvingAccounts', 'OpenRevolvingMonthlyPayment',
           'CurrentDelinquencies', 'AmountDelinquent', 'DelinquenciesLast7Years', 'RevolvingCreditBalance',
           'BankcardUtilization', 'DebtToIncomeRatio', 'StatedMonthlyIncome', 'LoanNumber', 'LoanOriginalAmount',
           'LoanOriginationQuarter', 'MemberKey', 'MonthlyLoanPayment']
loans = loans[columns]

Some of the column headings are confusing or difficult to work with so I will rename them.

In [5]:
loans = loans.rename(columns={'ListingCategory (numeric)':'ListingCategory','IsBorrowerHomeowner':'HomeOwner',
                              'TotalCreditLinespast7years':'TotalCreditLines',
                              'DelinquenciesLast7Years':'TotalDelinquencies'})

I am interested in the effect of the features on Loan Outcome Status. Loans that have not yet reached an outcome need to be removed as I do not know what their final outcome will be (I cannot identify what effect their features have on Loan Outcome Status as their status is not yet known). First I will check the values in the LoanStatus column.

In [6]:
loans.LoanStatus.value_counts()

Current                   56576
Completed                 38074
Chargedoff                11992
Defaulted                  5018
Past Due (1-15 days)        806
Past Due (31-60 days)       363
Past Due (61-90 days)       313
Past Due (91-120 days)      304
Past Due (16-30 days)       265
FinalPaymentInProgress      205
Past Due (>120 days)         16
Cancelled                     5
Name: LoanStatus, dtype: int64

For this analysis I am going to consider 'Chargedoff' and 'Defaulted' as loans that were not completed successfully. Loans that are 'Past Due...' I will consider current.The loans to be removed are those listed as 'Current', 'Past Due...' and 'FinalPaymentInProgress'. 

In [7]:
# Remove loans that have not reached an outcome
loans = loans[loans['LoanStatus'].isin(['Completed', 'Chargedoff', 'Defaulted', 'Cancelled'])]
loans.LoanStatus.value_counts()

Completed     38074
Chargedoff    11992
Defaulted      5018
Cancelled         5
Name: LoanStatus, dtype: int64

I am interested in knowing the outcome of the loan - if it was successfully paid off or not. This will be easiest by creating a new column that records the final status of the loan - LoanCompleted - which holds a bool, either True (paid off) or False (not paid off).

In [8]:
# Return loan completion status
def getLoanCompleted(loan_status):
    """Return True if loan completed, False otherwise."""
    if loan_status.strip() == 'Completed':
        return True
    else:
        return False

In [9]:
# Create column to hold status of loan outcome
loans['LoanCompleted'] = loans['LoanStatus'].apply(getLoanCompleted)

In [10]:
# Examine remaining columns
loans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55089 entries, 0 to 113935
Data columns (total 27 columns):
Term                           55089 non-null int64
LoanStatus                     55089 non-null object
BorrowerAPR                    55064 non-null float64
BorrowerRate                   55089 non-null float64
ListingCategory                55089 non-null int64
EmploymentStatus               52834 non-null object
EmploymentStatusDuration       47474 non-null float64
HomeOwner                      55089 non-null bool
CreditScoreRangeLower          54498 non-null float64
CreditScoreRangeUpper          54498 non-null float64
CurrentCreditLines             47485 non-null float64
TotalCreditLines               54392 non-null float64
OpenRevolvingAccounts          55089 non-null int64
OpenRevolvingMonthlyPayment    55089 non-null float64
CurrentDelinquencies           54392 non-null float64
AmountDelinquent               47467 non-null float64
TotalDelinquencies             54099 

Several of the columns will need some cleaning before we can perform analysis on them.

In [11]:
# Convert LoanNumber to a string as we will not perform calcluations on it
loans['LoanNumber'] = loans['LoanNumber'].astype('str')

In [12]:
# Convert the LoanStatus column to categorical types
loans['LoanStatus'] = loans['LoanStatus'].astype('category')

In [13]:
# Convert the ListingCategory column to readable values
def convert_listing_category(cat):
    """Convert the listing category to a human-readable string."""
    categories = ['Not Available', 'Debt Consolidation', 'Home Improvement', 'Business', 'Personal Loan', 'Student Use',
                 'Auto', 'Other', 'Baby and Adoption', 'Boat', 'Cosmetic Procedure', 'Engagement Ring', 'Green Loans',
                 'Household Expense', 'Large Purchases', 'Medical and Dental', 'Motorcycle', 'RV', 'Taxes', 'Vacation',
                 'Wedding']
    return categories[cat]

loans['ListingCategory'] = loans['ListingCategory'].apply(convert_listing_category)

In [14]:
# Convert ListingCategoy to categorical types
loans['ListingCategory'] = loans['ListingCategory'].astype('category')

In [15]:
# Convert null EmploymentStatus to 'Not Available'
loans['EmploymentStatus'] = loans['EmploymentStatus'].fillna('Not available')

In [16]:
# Convert EmploymentStatus to categorical type
loans['EmploymentStatus'] = loans['EmploymentStatus'].astype('category')

In [17]:
# Convert LoanOriginationQuarter to categorical type
loans['LoanOriginationQuarter'] = loans['LoanOriginationQuarter'].astype('category')

### Note: Determine if drop missing values or provide a default value

In [18]:
loans.Term.value_counts()

36    49861
60     3696
12     1532
Name: Term, dtype: int64

There are only three possible terms, so these are better represented as categories

In [19]:
# Convert Term to categorical type
loans['Term'] = loans['Term'].astype('category')

In [20]:
dupes = loans[loans['LoanNumber'].duplicated() == True]
dupes

,Term,LoanStatus,BorrowerAPR,BorrowerRate,ListingCategory,EmploymentStatus,EmploymentStatusDuration,HomeOwner,CreditScoreRangeLower,CreditScoreRangeUpper,...,RevolvingCreditBalance,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanNumber,LoanOriginalAmount,LoanOriginationQuarter,MemberKey,MonthlyLoanPayment,LoanCompleted
33959,60,Completed,0.28875,0.2624,Business,Employed,6.0,False,660.0,679.0,...,8575.0,0.35,0.33,4506.666667,131358,5500,Q1 2014,85D535659273197054A0BB3,165.45,True
37956,36,Completed,0.22875,0.1915,Debt Consolidation,Employed,37.0,False,660.0,679.0,...,15237.0,0.42,0.20,6083.333333,105288,20000,Q4 2013,8EC335867884136891E9E14,734.64,True
51079,36,Completed,0.23438,0.1970,Debt Consolidation,Employed,222.0,True,660.0,679.0,...,38936.0,0.85,0.16,14583.333333,123094,5000,Q4 2013,9FA83698836314377793393,185.05,True
60004,36,Completed,0.13799,0.1099,Other,Employed,95.0,True,680.0,699.0,...,8171.0,0.37,0.23,4250.000000,110303,2000,Q4 2013,4553358888719707373CD1C,65.47,True
61289,36,Completed,0.09000,0.0766,Debt Consolidation,Self-employed,18.0,True,800.0,819.0,...,1091.0,0.04,NaN,7500.000000,107275,3500,Q4 2013,34193560492599576667729,109.13,True
68585,36,Completed,0.18633,0.1500,Debt Consolidation,Employed,182.0,False,700.0,719.0,...,13681.0,0.22,0.34,7500.000000,121979,15000,Q4 2013,27FA3533700778342A3BB33,519.98,True
70435,60,Completed,0.19988,0.1760,Debt Consolidation,Employed,78.0,True,720.0,739.0,...,4779.0,0.23,0.26,4125.000000,119188,15000,Q4 2013,4FC835464074587007C7ABC,377.64,True
96475,60,Completed,0.22601,0.2015,Debt Consolidation,Employed,81.0,False,680.0,699.0,...,15010.0,0.93,0.23,4125.000000,101908,15000,Q3 2013,7EF83383685864632D111F0,398.66,True
97137,36,Completed,0.32446,0.2850,Debt Consolidation,Employed,149.0,False,660.0,679.0,...,2306.0,0.62,0.17,5833.333333,121751,4000,Q4 2013,4E9335537538907782E9392,166.54,True
97330,60,Completed,0.23318,0.2085,Debt Consolidation,Employed,141.0,True,680.0,699.0,...,12124.0,0.78,0.32,4166.666667,118909,13000,Q4 2013,117636957815160767DC8DC,350.60,True


There are some duplicated records in the data (duplicated LoanNumber, ListingKey, ListingNumber). A visual check of these showed that the only difference between the duplicates was the ProsperScore, which is not being used in this analysis. For this reason we can drop the duplicate values.

In [21]:
# Drop rows duplicated on LoanNumber
loans.drop_duplicates('LoanNumber', inplace=True)

Now we can look at an overview of the remaining data of interest

In [22]:
print(loans.shape)
print(loans.dtypes)
print(loans.head(10))

(55076, 27)
Term                           category
LoanStatus                     category
BorrowerAPR                     float64
BorrowerRate                    float64
ListingCategory                category
EmploymentStatus               category
EmploymentStatusDuration        float64
HomeOwner                          bool
CreditScoreRangeLower           float64
CreditScoreRangeUpper           float64
CurrentCreditLines              float64
TotalCreditLines                float64
OpenRevolvingAccounts             int64
OpenRevolvingMonthlyPayment     float64
CurrentDelinquencies            float64
AmountDelinquent                float64
TotalDelinquencies              float64
RevolvingCreditBalance          float64
BankcardUtilization             float64
DebtToIncomeRatio               float64
StatedMonthlyIncome             float64
LoanNumber                       object
LoanOriginalAmount                int64
LoanOriginationQuarter         category
MemberKey                   

In [23]:
loans.describe()

,BorrowerAPR,BorrowerRate,EmploymentStatusDuration,CreditScoreRangeLower,CreditScoreRangeUpper,CurrentCreditLines,TotalCreditLines,OpenRevolvingAccounts,OpenRevolvingMonthlyPayment,CurrentDelinquencies,AmountDelinquent,TotalDelinquencies,RevolvingCreditBalance,BankcardUtilization,DebtToIncomeRatio,StatedMonthlyIncome,LoanOriginalAmount,MonthlyLoanPayment
count,55051.000000,55076.000000,47461.00000,54485.000000,54485.000000,47472.000000,54379.000000,55076.000000,55076.000000,54379.000000,47454.000000,54086.000000,4.747200e+04,47472.000000,50847.000000,55076.000000,55076.000000,55076.000000
mean,0.222197,0.200428,80.89071,671.735340,690.735340,9.568061,25.274977,6.077184,325.268356,0.906563,1051.079382,4.572385,1.569016e+04,0.530452,0.289868,5054.345827,6260.841383,223.700974
std,0.088179,0.081292,84.67440,81.917545,81.917545,5.582419,14.268204,4.529111,448.865776,2.596514,6677.685082,10.690307,3.401463e+04,0.350793,0.736789,5711.345824,5217.422427,185.723985
min,0.006530,0.000000,0.00000,0.000000,19.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1000.000000,0.000000
25%,0.149740,0.135000,21.00000,640.000000,659.000000,6.000000,15.000000,3.000000,60.000000,0.000000,0.000000,0.000000,1.613000e+03,0.210000,0.130000,2808.250000,2600.000000,98.220000
50%,0.214340,0.194500,52.00000,680.000000,699.000000,9.000000,23.000000,5.000000,183.000000,0.000000,0.000000,0.000000,6.071500e+03,0.560000,0.200000,4166.666667,4500.000000,172.600000
75%,0.295100,0.269900,112.00000,720.000000,739.000000,13.000000,33.000000,8.000000,418.000000,1.000000,0.000000,4.000000,1.617075e+04,0.850000,0.300000,6250.000000,8000.000000,300.387500
max,0.512290,0.497500,755.00000,880.000000,899.000000,59.000000,136.000000,51.000000,14985.000000,83.000000,444745.000000,99.000000,1.435667e+06,5.950000,10.010000,618547.833333,35000.000000,2251.510000


## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!